In [1]:
import zarr
import s3fs
import numpy as np
import xarray as xr

## Read Xarray with zarr

In [2]:
# running this twice fails
ds = xr.open_zarr('s3://hydromt-data/meteo/era5_daily.zarr', storage_options={"anon": False}, consolidated=False) # , decode_cf=False)
ds

<xarray.Dataset>
Dimensions:    (time: 26694, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1950-01-02 1950-01-03 ... 2023-02-01
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    msl        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tisr       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>

In [3]:
ds.close()  # does not seem to work

In [4]:
ds = xr.open_zarr('s3://hydromt-data/meteo/era5_daily.zarr', storage_options={"anon": False}, chunks="auto", consolidated=False) # , decode_cf=False)
ds  # this error happens at HydroMT level, somehow opening it twice

OverflowError: time values outside range of 64 bit signed integers

## Open source zarr GCS

In [9]:
ds_open = xr.open_zarr('gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2', chunks="auto", consolidated=True) # , decode_cf=False)
ds_open

<xarray.Dataset>
Dimensions:              (time: 1084032, values: 542080)
Coordinates:
    depthBelowLandLayer  float64 ...
    entireAtmosphere     float64 ...
    latitude             (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude            (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    number               int64 ...
    step                 timedelta64[ns] ...
    surface              float64 ...
  * time                 (time) datetime64[ns] 1900-01-01 ... 2023-08-31T23:0...
    valid_time           (time) datetime64[ns] dask.array<chunksize=(1084032,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables: (12/38)
    cape                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    d2m                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    hcc                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl1                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl2                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl3                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    ...                   ...
    tsn                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    u10                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    u100                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    v10                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    v100                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    z                    (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              1
    GRIB_subCentre:            0
    history:                   2023-08-27T15:04 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  89c3caaaaf19eb0f6e9749dbde6c93972d64419137e2c8...
    pangeo-forge:recipe_hash:  09c22f5fffc2fbe3742fbb38c8c2c761e1b41b50570f09...
    pangeo-forge:version:      0.9.5.dev2+gd43015b

## Write with zarr

In [2]:
url = "hydromt-data/notebooks"
fs = s3fs.S3FileSystem()
s3_store = zarr.storage.FSStore(url, normalize_keys=True, key_separator="/", mode="w", dimension_separator="/", fs=fs, check=False, create=False)

In [3]:
root = zarr.group(store=s3_store, overwrite=True)

In [4]:
z = root.zeros(name="zeros", shape=(100, 100), chunks=(2, 2), dtype='i4')
z.cdata_shape

(50, 50)

In [5]:
# Currently does not contain any metadata
dict(z.attrs)

{}

In [6]:
# zarr needs specific metadata: https://docs.xarray.dev/en/latest/internals/zarr-encoding-spec.html
z.attrs["_ARRAY_DIMENSIONS"] = ["x", "y"]
z.attrs["coordinates"] = "xc yc"
z.attrs["long_name"] = 'zeroes for testing',
# 'time_rep': 'instantaneous',
z.attrs["type_preferred"] = 'uint32',
# 'units': 'C'}

# Then create the dimension arrays
x = root.array("xc", np.arange(0, 100), chunks=(2,), dtype='i4')
x.attrs["_ARRAY_DIMENSIONS"] = ["x"]
x.attrs["long_name"] = "x"
y = root.array("yc", np.arange(0, 100), chunks=(2,), dtype='i4')
y.attrs["_ARRAY_DIMENSIONS"] = ["y"]
y.attrs["long_name"] = "y"

In [7]:
# change a single_value
z[50,50] = 42

In [8]:
# close the storage
s3_store.close()

In [18]:
# Because we use cloud storage, consolidate metadata
zarr.consolidate_metadata("s3://hydromt-data/notebooks")  # cannot use s3_store as argument, as the metadata are then placed under 's3://hydromt-data/notebooks//'

<zarr.hierarchy.Group '/'>

In [19]:
# read again
root = zarr.group(store=s3_store)
z = root['zeros']

In [20]:
z

<zarr.core.Array '/zeros' (100, 100) int32>

In [21]:
z[50,50]

42

In [22]:
z.chunks

(2, 2)

In [23]:
s3_store.close()

### read this using xr

In [28]:
ds_self = xr.open_zarr("s3://hydromt-data/notebooks")
ds_self

<xarray.Dataset>
Dimensions:  (x: 100, y: 100)
Coordinates:
    xc       (x) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    yc       (y) float64 dask.array<chunksize=(2,), meta=np.ndarray>
Dimensions without coordinates: x, y
Data variables:
    zeros    (x, y) float64 dask.array<chunksize=(2, 2), meta=np.ndarray>